In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, Reshape, Input, Conv2DTranspose, LeakyReLU, BatchNormalization, Dropout
from keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tqdm import tqdm
import seaborn as sns


# Google Drive'a bağlan
from google.colab import drive
drive.mount('/content/drive')

# Görüntüleri yükleme fonksiyonu
def load_images(folder):
    imgs = []
    target = 1
    labels = []
    for i in os.listdir(folder):
        img_dir = os.path.join(folder,i)
        try:
            img = cv2.imread(img_dir)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (128,128))
            imgs.append(img)
            labels.append(target)
        except:
            continue

    imgs = np.array(imgs)
    labels = np.array(labels)

    return imgs, labels

# Üretici Model
def build_generator():
    model = Sequential([
        Dense(32*32*256, input_dim=NOISE_DIM),
        LeakyReLU(alpha=0.2),
        Reshape((32,32,256)),

        Conv2DTranspose(128, (4, 4), strides=2, padding='same'),
        LeakyReLU(alpha=0.2),

        Conv2DTranspose(128, (4, 4), strides=2, padding='same'),
        LeakyReLU(alpha=0.2),

        Conv2D(CHANNELS, (4, 4), padding='same', activation='tanh')
    ],
    name="generator")
    model.summary()
    model.compile(loss="binary_crossentropy", optimizer=OPTIMIZER)
    return model

# Ayrıştırıcı Model
def build_discriminator():
    model = Sequential([
        Conv2D(64, (3, 3), padding='same', input_shape=(WIDTH, HEIGHT, CHANNELS)),
        LeakyReLU(alpha=0.2),

        Conv2D(128, (3, 3), strides=2, padding='same'),
        LeakyReLU(alpha=0.2),

        Conv2D(128, (3, 3), strides=2, padding='same'),
        LeakyReLU(alpha=0.2),

        Conv2D(256, (3, 3), strides=2, padding='same'),
        LeakyReLU(alpha=0.2),

        Flatten(),
        Dropout(0.4),
        Dense(1, activation="sigmoid")
    ], name="discriminator")
    model.summary()
    model.compile(loss="binary_crossentropy", optimizer=OPTIMIZER)
    return model

# GAN Model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(NOISE_DIM,))
    fake_image = generator(gan_input)
    gan_output = discriminator(fake_image)
    gan = Model(gan_input, gan_output, name="gan_model")
    gan.compile(loss="binary_crossentropy", optimizer=OPTIMIZER)
    return gan

# Fonksiyonunuz
def sample_images(generator, noise, subplots, figsize=(22, 8), save=False):
    generated_images = generator.predict(noise)
    plt.figure(figsize=figsize)

    for i, image in enumerate(generated_images):
        plt.subplot(subplots[0], subplots[1], i+1)
        if CHANNELS == 1:
            image = image.reshape((WIDTH, HEIGHT))
            plt.imshow(image, cmap='gray')
        else:
            image = image.reshape((WIDTH, HEIGHT, CHANNELS))
            plt.imshow(image)
        if save:
            img_name = "gen" + str(i)
            plt.savefig(img_name)
        plt.subplots_adjust(wspace=None, hspace=None)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Parametreler
NOISE_DIM = 100
BATCH_SIZE = 4
STEPS_PER_EPOCH = 3750
EPOCHS = 10
SEED = 42
WIDTH, HEIGHT, CHANNELS = 128, 128, 1
OPTIMIZER = Adam(0.0002, 0.5)
MAIN_DIR = "/content/drive/MyDrive/archive/MY_data/yes"

# Veri yükleme
data, labels = load_images(MAIN_DIR)
print("Data shape:", data.shape)
print("Labels shape:", labels.shape)

# Rastgele örnekler seç
np.random.seed(SEED)
idxs = np.random.randint(0, len(data), 20)
X_train = data[idxs]

# Verilerin normalizasyonu
data_normalized = (data.astype(np.float32) - 127.5) / 127.5

# Resimleri yeniden şekillendirme
X_train = data_normalized.reshape(-1, WIDTH, HEIGHT, CHANNELS)

# Görüntüleri göster
plt.figure(figsize=(20, 8))
for i in range(10):
    axs = plt.subplot(2, 5, i+1)
    plt.imshow(X_train[i], cmap="gray")
    plt.axis('off')
    axs.set_xticklabels([])
    axs.set_yticklabels([])
    plt.subplots_adjust(wspace=None, hspace=None)
plt.tight_layout()
plt.show()

# Giriş verisini yeniden şekillendirme
NOISE_DIM = 100

# Model oluşturma
discriminator = build_discriminator()
generator = build_generator()
gan = build_gan(generator, discriminator)

# Model özetlerini görüntüleme
print("Discriminator Summary:")
discriminator.summary()
print("\nGenerator Summary:")
generator.summary()

# Build optimizer for all trainable variables
optimizer = Adam(0.0002, 0.5)
optimizer.build(generator.trainable_variables + discriminator.trainable_variables)

gan.compile(loss="binary_crossentropy", optimizer=optimizer)

print("\nGAN Summary:")
gan.summary()

# Eğitim
real = np.ones((BATCH_SIZE, 1))
fake = np.zeros((BATCH_SIZE, 1))

for epoch in range(EPOCHS):
    for _ in tqdm(range(STEPS_PER_EPOCH)):
        # Rastgele gerçek resim seçme
        idx = np.random.randint(0, X_train.shape[0], BATCH_SIZE)
        real_images = X_train[idx]

        # Gürültüden sahte resimler üretme
        noise = np.random.normal(0, 1, (BATCH_SIZE, NOISE_DIM))
        fake_images = generator.predict(noise)

        # Ayrıştırıcıyı eğitme
        d_loss_real = discriminator.train_on_batch(real_images, real)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Üreticiyi eğitme
        noise = np.random.normal(0, 1, (BATCH_SIZE, NOISE_DIM))
        g_loss = gan.train_on_batch(noise, real)

    print(f"Epoch: {epoch+1}, Discriminator Loss: {d_loss}, Generator Loss: {g_loss}")

    # Her epoch sonunda örnek resimler üretme ve kaydetme
    noise = np.random.normal(0, 1, (10, NOISE_DIM))
    sample_images(generator, noise, subplots=(5, 5), figsize=(10, 10), save=True)



    # Sonuç görüntüleri üretme
noise = np.random.normal(0, 1, size=(100, NOISE_DIM))
sample_images(generator, noise, subplots=(10, 10), figsize=(24, 20), save=True)

generated_images = generator.predict(noise)
generated_images.shape

fig, axs = plt.subplots(ncols=1, nrows=1, figsize=(18,10))

sns.distplot(X_train, label='Real Images', hist=True, color='#fc0328', ax=axs)
sns.distplot(generated_images, label='Generated Images', hist=True, color='#0c06c7', ax=axs)

axs.legend(loc='upper right', prop={'size': 12})

plt.show()







Output hidden; open in https://colab.research.google.com to view.